In [1]:
import numpy as np
from numpy import log10
import matplotlib.pyplot as plt
%matplotlib qt5

import h5py

from workprojects.rfistats.filterbank import Filterbank
from workprojects.rfistats.block_stats import analyse_block
from workprojects.rfistats.filterbank_stats import FilterbankStats
from workprojects.rfistats.stats_utils import outlier_mask, robust_std

In [3]:
fstats = FilterbankStats.load_hdf5('/home/vmorello/repositories/workprojects/rfistats/SCP_stats.h5')

In [6]:
plt.figure(figsize=(30, 6), dpi=250)
plt.imshow(fstats.occupancy, aspect='auto', interpolation='nearest')
plt.xlabel('Frequency Channel Index')
plt.ylabel('Data Block Index')
plt.colorbar()
plt.tight_layout()
plt.savefig('/home/vmorello/Desktop/occupancy.png')
plt.close()

In [38]:
occ = fstats.time_average('avg_power')

plt.figure(figsize=(18, 9))
plt.plot(fstats.freqs, sensitivity)
plt.xlim(fstats.freqs[0], fstats.freqs[-1])
plt.tight_layout()
plt.grid()
plt.show()


In [11]:
occupancy = fstats.time_average('occupancy')
avg_power = fstats.time_average('avg_power')

std = robust_std(avg_power)
med = np.median(avg_power)
ymin = med - 5*std
ymax = med + 50 * std

abnormal_power = med + 5 * std


badmask = (occupancy > 0.01) | (avg_power > med + 5*std)
badchans = np.where(badmask)[0]

boundary_freqs = np.concatenate((fstats.freqs, [fstats.freqs[-1] + fstats.df])) - fstats.df/2.0
boundary_freqs_badmask = np.zeros(boundary_freqs.size, dtype=bool)
#boundary_freqs_badmask[badchans-1] = True 
boundary_freqs_badmask[badchans] = True 
boundary_freqs_badmask[badchans+1] = True 

plt.figure(figsize=(36,6))

ax1 = plt.gca()
ax1.plot(fstats.freqs, occupancy, color='b', alpha=0.5, label='Occupancy')
ax1.set_ylabel('Occupancy', color='b', alpha=0.5)
ax1.tick_params('y', colors='b')

ax2 = ax1.twinx()
ax2.plot(fstats.freqs, avg_power, marker='.', markersize=3, color='r', alpha=0.5, label='Average Normalised power')
ax2.set_ylabel('Average Normalised Power', color='r', alpha=0.5)
ax2.tick_params('y', colors='r')
ax2.plot([fstats.freqs[0], fstats.freqs[-1]], [abnormal_power, abnormal_power], 'k--', label='Median channel power + 5 sigma')
ax2.fill_between(boundary_freqs, boundary_freqs_badmask*ymax, ymin,  color='k', alpha=0.3, label='Bad Channels: power above limit and occupancy > 1%')
ax2.set_ylim(ymin, ymax)



ax1.set_xlabel('Frequency (MHz)')
plt.xlim(fstats.freqs[0], fstats.freqs[-1])

plt.legend()



plt.tight_layout()
plt.savefig('/home/vmorello/Desktop/rfistats_plots/bad_channels.png')
plt.close()

In [54]:
np.corrcoef(stats['avg_power'].mean(axis=0), stats['occupancy'].mean(axis=0))

array([[ 1.        ,  0.15212338],
       [ 0.15212338,  1.        ]])

In [38]:
fil.nsamp // 1634

480

In [63]:

mask = (stats['occupancy'].mean(axis=0) > 1e-4) & (stats['avg_power'].mean(axis=0) > 1.5)

plt.figure(figsize=(18, 9))
plt.bar(range(mask.size), mask, width=1)
plt.tight_layout()
plt.show()


In [27]:
fstats.robust_std

array([[ 155.62111025,   14.43799739,   13.84727307, ...,   31.32576329,
          32.26976392,   32.83732258],
       [ 156.64039928,   14.06155542,   14.4032489 , ...,   30.2833086 ,
          31.54583706,   32.32767807],
       [ 158.56314903,   14.2874206 ,   13.87043873, ...,   32.44929778,
          32.64041448,   30.93773849],
       ..., 
       [ 157.88555349,   14.09630391,   14.20634079, ...,   30.97827839,
          29.94161512,   31.93965327],
       [ 160.69438972,   14.2874206 ,   14.06155542, ...,   32.50142052,
          32.92998522,   32.21764119],
       [ 151.68873952,   14.22950645,   14.60594842, ...,   30.61341926,
          31.63270828,   32.84890541]])